데이터 10000개 최종버전

In [ ]:
from transformers import (
    Seq2SeqTrainer, Seq2SeqTrainingArguments, EarlyStoppingCallback,
    AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, GenerationConfig
)
import evaluate
import numpy as np
import torch
from google.colab import files
import json, os, shutil
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'evaluate'

In [ ]:
SAVE_DIR = "./results"
LOG_DIR  = "./logs"
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)ㅁ
print("MODEL SAVE:", os.path.abspath(SAVE_DIR))
print("LOG DIR   :", os.path.abspath(LOG_DIR))

MODEL SAVE: /content/results
LOG DIR   : /content/logs


In [ ]:
MODEL_NAME = "wisenut-nlp-team/KoT5-base"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_korean(example):
    ctx = [c.strip() for c in example.get("context", [])[:-1]]
    last = example.get("context", [""])[-1].strip() if example.get("context") else ""
    text = " ".join(ctx + [last]).strip()
    target = str(example.get("output","")).strip()
    return {"input_text": text, "target_text": target}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [ ]:
try:
    uploaded = files.upload()
    file_path = next(f for f in uploaded if f.endswith(".jsonl"))
    with open(file_path, "r", encoding="utf-8") as f:
        raw_data = [json.loads(line) for line in f if line.strip()]
except Exception as e:
    print(f"파일 업로드 중 오류 발생: {e}")
    # 예시 데이터 생성
    raw_data = [
        {"context": ["안녕"], "output": "안녕하세요."},
        {"context": ["너 이름이 뭐니?"], "output": "성함이 어떻게 되세요?"}
    ]

raw_data_clean = [
    r for r in raw_data
    if r.get("context") and str(r["context"][-1]).strip() and str(r.get("output","")).strip()
]

processed = [preprocess_korean(d) for d in raw_data_clean]
print(f"원본 {len(raw_data)} → 정제 {len(processed)}")

train_list, test_list = train_test_split(processed, test_size=0.1, random_state=42)

class RewriteDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.data)
    def __getitem__(self, i):
        item = self.data[i]
        enc = self.tokenizer(
            item["input_text"], max_length=self.max_len, truncation=True, padding=False
        )
        lbl = self.tokenizer(
            text_target=item["target_text"], max_length=self.max_len, truncation=True, padding=False
        )
        return {
            "input_ids": enc.input_ids,
            "attention_mask": enc.attention_mask,
            "labels": lbl.input_ids,
        }

train_ds = RewriteDataset(train_list, tokenizer, max_len=384)
test_ds  = RewriteDataset(test_list,  tokenizer, max_len=384)

Saving 순화표현모델 데이터_리라이팅완료.jsonl to 순화표현모델 데이터_리라이팅완료.jsonl
원본 10012 → 정제 10008


In [ ]:
meteor_metric    = evaluate.load("meteor")
bertscore_metric = evaluate.load("bertscore")

RNG = np.random.RandomState(1234)
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, torch.Tensor):  preds  = preds.detach().cpu().numpy()
    if isinstance(labels, torch.Tensor): labels = labels.detach().cpu().numpy()

    preds  = np.where(preds  == -100, tokenizer.pad_token_id, preds)
    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)

    dec_preds  = tokenizer.batch_decode(preds, skip_special_tokens=True)
    dec_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    meteor = meteor_metric.compute(predictions=dec_preds, references=dec_labels)["meteor"]

    # BERTScore: 샘플링해 속도 확보
    if len(dec_preds) > 300:
        idx = RNG.choice(len(dec_preds), size=300, replace=False)
        p_s, r_s = [dec_preds[i] for i in idx], [dec_labels[i] for i in idx]
    else:
        p_s, r_s = dec_preds, dec_labels
    bert_f1 = float(np.mean(bertscore_metric.compute(predictions=p_s, references=r_s, lang="ko")["f1"]))

    return {"meteor": meteor, "bertscore": bert_f1}

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
from transformers import GenerationConfig

def model_init():
    m = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)

    if tokenizer.pad_token_id is None:
        tokenizer.add_special_tokens({"pad_token": "<pad>"})
        m.resize_token_embeddings(len(tokenizer))

    if m.config.decoder_start_token_id is None:
        m.config.decoder_start_token_id = tokenizer.pad_token_id
    if m.config.bos_token_id is None:
        m.config.bos_token_id = tokenizer.pad_token_id

    if m.config.eos_token_id is None and tokenizer.eos_token_id is not None:
        m.config.eos_token_id = tokenizer.eos_token_id

    m.generation_config = GenerationConfig.from_model_config(m.config)
    m.generation_config.num_beams = 5
    m.generation_config.max_new_tokens = 64
    m.generation_config.min_new_tokens = 8
    m.generation_config.no_repeat_ngram_size = 3
    m.generation_config.early_stopping = True
    m.generation_config.length_penalty = 1.0

    return m.to(device)

In [ ]:
use_bf16 = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8

FIXED_ARGS = Seq2SeqTrainingArguments(
    output_dir=SAVE_DIR,
    logging_dir=LOG_DIR,
    learning_rate=2.695678727689929e-05,         # ✔ lr
    num_train_epochs=5,                           # ✔ epochs
    per_device_train_batch_size=4,                # ✔ batch size
    weight_decay=0.046196492892156577,            # ✔ weight decay
    label_smoothing_factor=0.04137733503969081,   # ✔ label smoothing
    warmup_steps=353,                              # ✔ warmup steps
    lr_scheduler_type="linear",

    evaluation_strategy="epoch",                  # ✔ 올바른 파라미터명
    save_strategy="epoch",
    predict_with_generate=True,
    metric_for_best_model="bertscore",
    greater_is_better=True,
    logging_steps=50,
    load_best_model_at_end=True,
    report_to="none",
    bf16=use_bf16,
    fp16=False,                                   # bf16 지원 GPU면 bf16만 켜는 게 안전
    max_grad_norm=1.0,
    remove_unused_columns=False,
)

In [ ]:
# === Trainer 구성 (model_init 한 번만 사용) ===
model = model_init()
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=FIXED_ARGS,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

# === 학습 ===
trainer.train()

# === 저장 ===
trainer.save_model(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

# === ZIP & 다운로드(선택) ===
zip_path = 'trained_model.zip'
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', SAVE_DIR)

from google.colab import files
files.download(zip_path)
print(f"\n✅ {zip_path} 다운로드 완료")

[I 2025-08-12 04:30:26,375] A new study created in memory with name: no-name-2a3a4082-f65a-4915-bcac-90579477fe24
/tmp/ipython-input-1950890406.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Rougel,Meteor,Bertscore
1,4.167200,3.983673,0.005328,0.040171,0.702033
2,3.449400,3.310675,0.002664,0.045820,0.705812
3,3.239700,3.093219,0.003330,0.046883,0.709708


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


[I 2025-08-12 04:52:10,070] Trial 0 finished with value: 0.7075922012329101 and parameters: {'learning_rate': 7.490206736019995e-06, 'num_train_epochs': 3, 'per_device_train_batch_size': 4, 'weight_decay': 0.033269473715652434, 'label_smoothing_factor': 0.14178672015314173, 'warmup_steps': 213}. Best is trial 0 with value: 0.7075922012329101.
/tmp/ipython-input-1950890406.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Rougel,Meteor,Bertscore
1,3.789300,3.577909,0.003663,0.044640,0.708610
2,2.659900,2.436146,0.986680,0.292325,0.764788
3,2.404600,2.334856,0.986680,0.297651,0.768805
4,2.536200,2.279349,0.986846,0.307761,0.773941
5,2.502200,2.235238,0.986014,0.322110,0.779334
6,2.154500,2.208965,0.986513,0.330155,0.782442
7,2.199100,2.188029,0.985598,0.341708,0.785854
8,2.100100,2.175674,0.985847,0.351354,0.784452
9,2.109100,2.166773,0.986180,0.357617,0.788302
10,2.181500,2.165265,0.986180,0.359404,0.792528


...


🎉 최적 파라미터: {'learning_rate': 2.695678727689929e-05, 'num_train_epochs': 5, 'per_device_train_batch_size': 4, 'weight_decay': 0.046196492892156577, 'label_smoothing_factor': 0.04137733503969081, 'warmup_steps': 353}
최고 BERTScore: 0.7976


/tmp/ipython-input-1057347936.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  final_trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Rougel,Meteor,Bertscore
1,2.583000,2.352252,0.986680,0.295127,0.768886
2,2.309600,2.203168,0.986813,0.331724,0.775022
3,2.035200,2.131358,0.987346,0.366423,0.786485
4,2.198500,2.107321,0.986846,0.380673,0.793971
5,2.177700,2.101067,0.987013,0.379977,0.795451


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import torch
from transformers import AutoTokenizer, BertForSequenceClassification

# === 경로 확정 ===
MODEL_PATH = os.path.abspath("/content/drive/MyDrive/trained_model")
print("모델 경로:", MODEL_PATH, "| 존재 여부:", os.path.exists(MODEL_PATH))

# === 디바이스 ===
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === 토크나이저는 허브에서 로드 ===
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert", trust_remote_code=True)

# === 모델은 로컬에서 로드 ===
model = BertForSequenceClassification.from_pretrained(MODEL_PATH, trust_remote_code=True).to(DEVICE)
model.eval()

print("✅ 모델 + 토크나이저 로드 완료")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
모델 경로: /content/drive/MyDrive/trained_model | 존재 여부: True


You are using a model of type t5 to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory /content/drive/MyDrive/trained_model.

In [ ]:
model = final_trainer.model
def generate_rewrite(model, tokenizer, context, input_text):
    prompt = f"[REWRITE]\n" + "\n".join(context) + f"\n{input_text}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)

    output_ids = model.generate(
        **inputs,
        max_length=64,
        num_beams=5,
        early_stopping=True
    )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 1. 채팅 입력 받기
chat_history = []
print("대화 맥락을 줄 단위로 입력하세요. 마지막에 빈 줄(Enter) 한 번 더 치면 입력이 종료됩니다.")
while True:
    line = input()
    if line.strip() == "":
        break
    chat_history.append(line)

# 2. 전체 채팅 출력
print("\n입력한 전체 채팅:")
for idx, line in enumerate(chat_history):
    print(f"{idx}: {line}")

# 3. 리라이팅할 문장 선택
target_idx = int(input("\n리라이팅할 문장의 번호를 입력하세요: "))
target_input = chat_history[target_idx]

# 4. 모델 입력 생성
context = chat_history[:target_idx]
input_text = f"{chat_history[target_idx]}"

# 5. 모델 추론
prompt = "[REWRITE]\n" + "\n".join(context) + f"\n{input_text}"
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)

output_ids = model.generate(
    **inputs,
    max_length=64,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.9,
)
generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 6. 결과 출력
print("\n🎯 리라이팅 결과:")
print("원문:", input_text)
print("공손하게 바꾼 문장:", generated)

채팅을 입력하세요. 입력을 마치면 빈 줄(엔터)로 종료합니다.
A: 아 오늘 지각했어
B: 또?
A: 오늘 아침에 밥먹고 오느라
B: 너같이 시간약속 안지키는 애들은 걍 맞아야함ㅋㅋ


입력한 전체 채팅:
0: A: 아 오늘 지각했어
1: B: 또?
2: A: 오늘 아침에 밥먹고 오느라
3: B: 너같이 시간약속 안지키는 애들은 걍 맞아야함ㅋㅋ

리라이팅할 문장의 번호를 입력하세요: 3


Both `max_new_tokens` (=64) and `max_length`(=64) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



🎯 리라이팅 결과:
원문: B: 너같이 시간약속 안지키는 애들은 걍 맞아야함ㅋㅋ
공손하게 바꾼 문장: B: 약속 지키느라 힘들었겠다


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 1) 디바이스 및 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FINETUNED_DIR = "/content/kot5-2step"   # 실제 모델 경로로 수정
tokenizer = AutoTokenizer.from_pretrained(FINETUNED_DIR)
model     = AutoModelForSeq2SeqLM.from_pretrained(FINETUNED_DIR).to(device)

# 2) 입력 받기
print("대화 맥락을 줄 단위로 입력하세요. 마지막에 빈 줄(Enter) 한 번 더 치면 입력이 종료됩니다.")
lines = []
while True:
    line = input()
    if not line.strip():
        break
    lines.append(line.strip())

if len(lines) < 2:
    raise ValueError("최소 2줄 이상의 대화(맥락+순화 대상)가 필요합니다.")

# 3) 포맷팅 (Step 1 영어 프롬프트 기준)
prefix = "rewrite last line politely:"
input_text = " ".join([prefix] + lines)

# 4) 생성
inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to(device)
outputs = model.generate(
    **inputs,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)

# 5) 결과 출력
print("\n=== 입력 대화 ===")
for l in lines:
    print(l)
print("\n=== 순화 결과 ===")
print(result)